In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

In [12]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [13]:
print(train_data.shape, test_data.shape)
print(train_targets.shape)

print(train_data[0])
print(train_targets[0])

print(train_data[1])
print(train_targets[1])

print(train_data[2])
print(train_targets[2])

(404, 13) (102, 13)
(404,)
[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
15.2
[2.1770e-02 8.2500e+01 2.0300e+00 0.0000e+00 4.1500e-01 7.6100e+00
 1.5700e+01 6.2700e+00 2.0000e+00 3.4800e+02 1.4700e+01 3.9538e+02
 3.1100e+00]
42.3
[4.89822e+00 0.00000e+00 1.81000e+01 0.00000e+00 6.31000e-01 4.97000e+00
 1.00000e+02 1.33250e+00 2.40000e+01 6.66000e+02 2.02000e+01 3.75520e+02
 3.26000e+00]
50.0


## Data
여기서 볼 수 있듯이 404개의 훈련 샘플과 102개의 테스트 샘플이 있고 모두 13개의 수치 특성을 가지고 있습니다. 13개의 특성은 다음과 같습니다:

1. Per capita crime rate.
2. Proportion of residential land zoned for lots over 25,000 square feet.
3. Proportion of non-retail business acres per town.
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. Nitric oxides concentration (parts per 10 million).
6. Average number of rooms per dwelling.
7. Proportion of owner-occupied units built prior to 1940.
8. Weighted distances to five Boston employment centres.
9. Index of accessibility to radial highways.
10. Full-value property-tax rate per $10,000.
11. Pupil-teacher ratio by town.
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13. % lower status of the population.

타깃은 주택의 중간 가격으로 천달러 단위입니다:

## Data Normalization 

In [14]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

print(train_data[0])
print(train_data[1])
print(train_data[2])

[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
  0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
  0.8252202 ]
[-0.40342651  2.99178419 -1.33391162 -0.25683275 -1.21518188  1.89434613
 -1.91036058  1.24758524 -0.85646254 -0.34843254 -1.71818909  0.43190599
 -1.32920239]
[ 0.1249402  -0.48361547  1.0283258  -0.25683275  0.62864202 -1.82968811
  1.11048828 -1.18743907  1.67588577  1.5652875   0.78447637  0.22061726
 -1.30850006]


## Build Model

In [18]:
from keras import models
from keras import layers

def build_model():
    # We will generate same model
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

* 마지막 Layer는 Activation function이 없습니다. 회귀 문제(어떤 값을 예측하는 문제)를 풀때 이와 같이 구성합니다.
* 평균 제곱 오차 (mean squared error)
* 절대 평균 오차 (mean absolute error)
* 0.5 indicate $500

## K-Fold Cross Validation

In [24]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=10, verbose=1)
    
    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)


처리중인 폴드 # 0
Epoch 1/100
303/303 [==============================] - 0s 990us/step - loss: 482.3604 - mean_absolute_error: 20.0016
Epoch 2/100
303/303 [==============================] - 0s 125us/step - loss: 284.2223 - mean_absolute_error: 14.5663
Epoch 3/100
303/303 [==============================] - 0s 138us/step - loss: 106.5324 - mean_absolute_error: 8.1621
Epoch 4/100
303/303 [==============================] - 0s 122us/step - loss: 45.0409 - mean_absolute_error: 5.0695
Epoch 5/100
303/303 [==============================] - 0s 152us/step - loss: 31.7193 - mean_absolute_error: 4.1844
Epoch 6/100
303/303 [==============================] - 0s 165us/step - loss: 26.2050 - mean_absolute_error: 3.6606
Epoch 7/100
303/303 [==============================] - 0s 152us/step - loss: 23.0923 - mean_absolute_error: 3.4543
Epoch 8/100
303/303 [==============================] - 0s 135us/step - loss: 20.6307 - mean_absolute_error: 3.1924
Epoch 9/100
303/303 [==============================] - 0s 122us

In [25]:
all_scores

[2.3235703270034036, 2.324590843502838, 2.613592069928009, 2.5778523825182775]

In [26]:
np.mean(all_scores)

2.4599014057381323

In [27]:
# 아래 모델은 k fold cross validation에서 마지막에 훈련된 모델입니다.
# 본 실습에서 별도의 test data set을 구성하지 않았기 때문에, vadliation data set을 이용해 house price 를 예측prediction 해보겠습니다.

predictedHousePrice = model.predict(val_data)

In [28]:
# validation set 은 100개의 데이터 셋으로 구성되어 있습니다만 아래와 같이 1대1 비교를 해보겠습니다.
# val_data[0] 에 대한 예측값 predictedHousePrice[0]
# val_data[0] 에 대한 실제값 val_targets[0]
print (predictedHousePrice[0], val_targets[0])

[25.742985] 36.2
